In [11]:
import gurobipy as gp
from gurobipy import GRB

# Parameter
P = 0
p = -6
set_l = range(p,P+1)
# Create a new Gurobi model
model = gp.Model("optimization_problem")

# Define the continuous variables
x1 = model.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x1")
x2 = model.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x2")
w12 = model.addVar(vtype=GRB.CONTINUOUS, name="w12")
model.update()
Delta_w12 = model.addVar(lb=0, ub=1.5 * x1.UB, vtype=GRB.CONTINUOUS, name="Delta_w12")  # 1.5 * 1.5 upper bound
Delta_x1 = model.addVar(lb=0, ub=1.5 * 10**p, vtype=GRB.CONTINUOUS, name="Delta_x1")

# Define the indexed continuous variables (hat_x_k) and binary variables (z_k)
hat_x = model.addVars(10, vtype=GRB.CONTINUOUS, name="hat_x")
z = model.addVars(10, vtype=GRB.BINARY, name="z")

# Set the objective function: Minimize f = -x1 + w12 - x2
model.setObjective(-x1 + w12 - x2, GRB.MINIMIZE)

# Add constraints in the order provided in the image

# Constraint 1: -6*x1 + 8*x2 <= 3
model.addConstr(-6*x1 + 8*x2 <= 3, "c1")

# Constraint 2: 3*x1 - x2 <= 3
model.addConstr(3*x1 - x2 <= 3, "c2")

# Constraint 3: w12 = sum(k * hat_x[k]) + Delta_w12
model.addConstr(w12 == gp.quicksum(k * hat_x[k] for k in range(10)) + Delta_w12, "c3")

# Constraint 4: x1 = sum(10^0 * k * z_k) + Delta_x1
model.addConstr(x1 == gp.quicksum(10**0 * k * z[k] for k in range(10)) + Delta_x1, "c4")

# Constraint 5: x2 = sum(hat_x[k])
model.addConstr(x2 == gp.quicksum(hat_x[k] for k in range(10)), "c5")

# Constraint 6: hat_x[k] <= 1.5 * z[k] for all k
for k in range(10):
    model.addConstr(hat_x[k] <= 1.5 * z[k], f"c6_{k}")

# Constraint 7: Sum(z_k) = 1
model.addConstr(gp.quicksum(z[k] for k in range(10)) == 1, "c7")

# Constraint 8: 0 <= Delta_w12 <= 1.5 * Delta_x1
model.addConstr(Delta_w12 >= 0, "c8a")
model.addConstr(Delta_w12 <= 1.5 * Delta_x1, "c8b")

# Constraint 9: (x2 - 1.5) * 10^0 + 1.5 * Delta_x1 <= Delta_w12 <= x2 * 10^0
model.addConstr((x2 - 1.5) * 10**0 + 1.5 * Delta_x1 <= Delta_w12, "c9a")
model.addConstr(Delta_w12 <= x2 * 10**0, "c9b")

# Constraint 10: 0 <= Delta_x1 <= 1.5 * 10^0
model.addConstr(Delta_x1 >= 0, "c10a")
model.addConstr(Delta_x1 <= 1.5 * 10**0, "c10b")

# Constraint 11: 0 <= hat_x[k] <= 1.5 for all k
for k in range(10):
    model.addConstr(hat_x[k] >= 0, f"c11a_{k}")
    model.addConstr(hat_x[k] <= 1.5, f"c11b_{k}")

# Optimize the model
model.Params.OutputFlag = 1
model.optimize()

# # Display the results
# if model.status == GRB.OPTIMAL:
#     print(f"Optimal objective value: {model.ObjVal}")
#     print(f"x1 = {x1.X}")
#     print(f"x2 = {x2.X}")
#     print(f"w12 = {w12.X}")
#     print(f"Delta_w12 = {Delta_w12.X}")
#     print(f"Delta_x1 = {Delta_x1.X}")
#     for k in range(10):
#         print(f"hat_x[{k}] = {hat_x[k].X}")
#         print(f"z[{k}] = {z[k].X}")
# else:
#     print("No optimal solution found.")
print([i for i in set_l])

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 42 rows, 25 columns and 97 nonzeros
Model fingerprint: 0xbff4f3bc
Variable types: 15 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-06, 2e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 38 rows and 21 columns
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 12 nonzeros
Variable types: 3 continuous, 1 integer (1 binary)
Found heuristic solution: objective -0.3750026
Found heuristic solution: objective -1.0000015

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: -1 -0.375003 
No other solutions better than -1

Optimal solu